In [1]:
import pandas as pd

In [2]:
tr_log = pd.read_pickle('../data/train_log.pkl')
te_log = pd.read_pickle('../data/test_log00.pkl')

In [5]:
tr_log

,object_id,mjd,passband,flux,flux_err,detected,lumi,flux_ratio_sq,flux_by_flux_ratio_sq,date,year,month,3month,flux_norm1,flux_norm2
0,615,59750.421875,2,-544.810303,3.622952,1,NaN,22613.378906,-1.232000e+07,59750,0,0,0,-0.824688,0.841065
1,615,59750.429688,1,-816.434326,5.553370,1,NaN,21613.710938,-1.764618e+07,59750,0,0,0,-1.235849,0.429904
2,615,59750.437500,3,-471.385529,3.801213,1,NaN,15378.291016,-7.249104e+06,59750,0,0,0,-0.713543,0.952209
3,615,59750.445312,4,-388.984985,11.395031,1,NaN,1165.291626,-4.532809e+05,59750,0,0,0,-0.588812,1.076940
4,615,59752.406250,2,-681.858887,4.041204,1,NaN,28468.689453,-1.941163e+07,59752,0,0,0,-1.032140,0.633613
5,615,59752.414062,1,-1061.457031,6.472994,1,NaN,26890.224609,-2.854282e+07,59752,0,0,0,-1.606743,0.059009
6,615,59752.421875,3,-524.954590,3.552751,1,NaN,21833.025391,-1.146135e+07,59752,0,0,0,-0.794632,0.871121
7,615,59752.433594,4,-393.480225,3.599346,1,NaN,11950.845703,-4.702422e+06,59752,0,0,0,-0.595617,1.070136
8,615,59752.445312,5,-355.886780,10.421921,1,NaN,1166.079590,-4.149923e+05,59752,0,0,0,-0.538711,1.127041
9,615,59767.296875,2,-548.013550,3.462291,1,NaN,25052.751953,-1.372925e+07,59767,0,0,0,-0.829536,0.836216


In [3]:
pd.pivot_table(tr_log, index=['object_id', 'date'], columns=['passband'], values=['flux'])

flux                                                   \
passband                  0            1           2           3           4   
object_id date                                                                 
615       59750         NaN  -816.434326 -544.810303 -471.385529 -388.984985   
          59752         NaN -1061.457031 -681.858887 -524.954590 -393.480225   
          59767         NaN  -815.188599 -548.013550 -475.516052 -405.663818   
          59770         NaN  -820.042786 -554.903198 -477.004730 -400.270386   
          59779         NaN  -921.002502 -630.523682 -518.533997 -422.184509   
          59782         NaN  -449.095612 -280.039520 -316.704865 -332.885437   
          59797         NaN    35.511822  391.399231  330.623901  360.397858   
          59800         NaN   129.541901  168.739899   30.120724  -60.942333   
          59807         NaN  -420.796417 -256.660980 -298.936859 -311.977783   
          59810         NaN  -527.020325 -342.819763 -363.282532 -348.628662   
          59813         NaN -1100.440063 -678.045715 -506.687408 -304.049713   
          59819    6.878784          NaN         NaN         NaN         NaN   
          59820   39.364853          NaN         NaN         NaN         NaN   
          59821  -10.422381          NaN         NaN         NaN         NaN   
          59822  -65.485130          NaN         NaN         NaN         NaN   
          59823 -113.349159          NaN         NaN         NaN         NaN   
          59835         NaN  -178.149399  -54.949490 -140.818436 -200.294128   
          59839         NaN  -953.883728 -639.035950 -518.293274 -418.723907   
          59842         NaN -1003.971497 -502.215332 -233.167755  111.507675   
          59851  -68.502457          NaN         NaN         NaN         NaN   
          59854         NaN   217.894211  459.452667  361.023438  374.446442   
          59857         NaN   646.523193  599.812195  354.961365  293.879608   
          59864         NaN  -942.167908 -637.105347 -524.586548 -414.447723   
          59867         NaN  -910.677734 -332.763123  -62.065010  202.288223   
          59870         NaN   659.486694  604.344543  373.986511  322.604034   
          59873         NaN   -98.796974    4.656033  -93.732880 -165.793457   
          59874  -97.353195          NaN         NaN         NaN         NaN   
          59875  -97.523880          NaN         NaN         NaN         NaN   
          59876 -108.672577          NaN         NaN         NaN         NaN   
          59877 -116.913223          NaN         NaN         NaN         NaN   
...                     ...          ...         ...         ...         ...   
130779836 60464         NaN          NaN    3.294472         NaN         NaN   
          60466  113.988297          NaN         NaN         NaN         NaN   
          60490         NaN          NaN   12.464979         NaN         NaN   
          60491         NaN   -13.734149         NaN         NaN         NaN   
          60496         NaN          NaN  -14.478923         NaN         NaN   
          60497         NaN          NaN         NaN   12.178056         NaN   
          60498         NaN          NaN         NaN  -20.815857         NaN   
          60499   54.361107          NaN         NaN         NaN         NaN   
          60503         NaN          NaN         NaN    9.912425         NaN   
          60505         NaN          NaN   17.719660         NaN         NaN   
          60506         NaN          NaN         NaN         NaN         NaN   
          60510         NaN          NaN         NaN         NaN  -39.215195   
          60515         NaN          NaN         NaN         NaN         NaN   
          60524  136.082718          NaN         NaN         NaN         NaN   
          60525   43.910740          NaN         NaN         NaN         NaN   
          60526         NaN          NaN         NaN   59.376686         NaN   
          60528        

In [4]:
pd.pivot_table(te_log, index=['object_id', 'date'], columns=['passband'], values=['flux'])

flux                                             \
passband                 0         1          2          3          4   
object_id date                                                          
1100      59798        NaN  0.169314   2.540578   1.201880   0.198283   
          59801        NaN  0.313417   0.209669   1.592229   0.401574   
          59818   1.827003       NaN        NaN        NaN        NaN   
          59819  -2.805669       NaN        NaN        NaN        NaN   
          59820   2.987135       NaN        NaN        NaN        NaN   
          59821   2.223504       NaN        NaN        NaN        NaN   
          59822  -0.110523       NaN        NaN        NaN        NaN   
          59823   1.440519       NaN        NaN        NaN        NaN   
          59826        NaN  0.246789   1.121269  -0.703057   3.193752   
          59842        NaN -0.295555  -0.157702   0.706918   0.116159   
          59851   3.258603       NaN        NaN        NaN        NaN   
          59854        NaN -0.488145   0.947356  -1.945823  -2.882406   
          59857        NaN  0.071713   0.795534   0.243145  -2.353324   
          59867        NaN -0.225719  -1.164272  -2.512633   3.102701   
          59870        NaN  0.484273  -0.247210   2.075477  -0.382085   
          59873        NaN  0.699803   0.778831   1.355760  -4.572034   
          59874   0.142377       NaN        NaN        NaN        NaN   
          59875   4.840622       NaN        NaN        NaN        NaN   
          59876   0.500252       NaN        NaN        NaN        NaN   
          59877  -2.568991       NaN        NaN        NaN        NaN   
          59878  -0.730302       NaN        NaN        NaN        NaN   
          59879   4.402680       NaN        NaN        NaN        NaN   
          59880   0.740040       NaN        NaN        NaN        NaN   
          59884        NaN -0.696369   0.446888  -0.119517   2.893057   
          59887        NaN -4.706995   0.311924   0.365026   0.613029   
          59896        NaN  0.662781  -0.474670   0.778466   0.818850   
          59899        NaN  0.438794  -0.531356  -1.016001   1.022785   
          59902        NaN  1.673340   0.970875  -2.289691   7.272311   
          59904   2.757962       NaN        NaN        NaN        NaN   
          59905   2.492357       NaN        NaN        NaN        NaN   
...                    ...       ...        ...        ...        ...   
130785900 60168        NaN       NaN  51.879932        NaN        NaN   
          60331        NaN       NaN        NaN        NaN        NaN   
          60334        NaN       NaN        NaN        NaN        NaN   
          60349        NaN       NaN        NaN        NaN -15.441579   
          60356        NaN       NaN  -2.230836        NaN        NaN   
          60361        NaN       NaN        NaN   3.889940        NaN   
          60363        NaN       NaN        NaN        NaN        NaN   
          60365        NaN       NaN        NaN        NaN  50.421410   
          60371        NaN       NaN        NaN  -5.473629        NaN   
          60376        NaN -0.846112        NaN        NaN        NaN   
          60380        NaN       NaN        NaN        NaN -22.747831   
          60381 -18.276602       NaN        NaN        NaN        NaN   
          60383        NaN       NaN  -0.086242        NaN        NaN   
          60386        NaN       NaN        NaN   0.795080        NaN   
          60388        NaN       NaN   3.104043        NaN        NaN   
          60389        NaN       NaN        NaN  10.263869        NaN   
          60393        NaN       NaN        NaN        NaN        NaN   
          60398        NaN       NaN        NaN        NaN        NaN   
          60400        NaN       NaN        NaN   7.484679        NaN   
          60414        NaN       NaN  -0.756714        NaN        NaN   
          60429        NaN       NaN        NaN   0.670120        NaN   
       